## 1. CoinGecko

In [7]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import time

In [8]:
cg = CoinGeckoAPI()

### 1.1 `cg.get_coin_market_chart_by_id()` API


- **:param `id`:**          default is Bitcoin "bitcoin"
- **:param `vs_currency`:** default is USD "usd"
- **:param `days`:**        the range of days to retireve data for "1", "30", "max", etc.
- **:param `interval`:**    the data interval "minutely", "hourly", "daily"

In [3]:
data = cg.get_coin_market_chart_by_id(id = "bitcoin",
                                      vs_currency = "usd",
                                      days = "5",
                                      interval = "daily")

###
# :param id:          default is Bitcoin "bitcoin"
# :param vs_currency: default is USD "usd"
# :param days:        the range of days to retireve data for "1", "30", "max", etc.
# :param interval:    the data interval "minutely", "hourly", "daily"
###

In [4]:
type(data)

dict

In [5]:
#data: a dictionary with 3 keys: prices, market_caps and total_volumes

#each key associates with a list, in which each element is also a list of two elements: timestamp and value of the key

data

{'prices': [[1729555200000, 67394.85775735618],
  [1729641600000, 67351.04793512498],
  [1729728000000, 66683.96806542306],
  [1729814400000, 68214.05205157402],
  [1729900800000, 66585.66535501445],
  [1729911829000, 66650.69451353882]],
 'market_caps': [[1729555200000, 1332326858574.4448],
  [1729641600000, 1331426858508.4004],
  [1729728000000, 1318285437546.2786],
  [1729814400000, 1347888109010.5889],
  [1729900800000, 1313975091942.221],
  [1729911829000, 1320159363921.0457]],
 'total_volumes': [[1729555200000, 40485496091.95008],
  [1729641600000, 31398389967.38113],
  [1729728000000, 32676918488.059],
  [1729814400000, 35892606570.03549],
  [1729900800000, 48426924850.053406],
  [1729911829000, 50304110341.034874]]}

In [6]:
data["prices"]

[[1729555200000, 67394.85775735618],
 [1729641600000, 67351.04793512498],
 [1729728000000, 66683.96806542306],
 [1729814400000, 68214.05205157402],
 [1729900800000, 66585.66535501445],
 [1729911829000, 66650.69451353882]]

In [7]:
df = pd.DataFrame(data["prices"], columns = ["Timestamp", "Price"])
df.head()

,Timestamp,Price
0,1729555200000,67394.857757
1,1729641600000,67351.047935
2,1729728000000,66683.968065
3,1729814400000,68214.052052
4,1729900800000,66585.665355


#### Convert Unix timestamp (in milliseconds) into standard date-time format

**`pd.to_datetime()`**

In [8]:
df["Date"] = pd.to_datetime(df["Timestamp"], unit="ms")

In [9]:
df.head()

,Timestamp,Price,Date
0,1729555200000,67394.857757,2024-10-22
1,1729641600000,67351.047935,2024-10-23
2,1729728000000,66683.968065,2024-10-24
3,1729814400000,68214.052052,2024-10-25
4,1729900800000,66585.665355,2024-10-26


#### Set the "Date” (date-time values) as the DataFrame's index

Many Pandas time-based operations (like `resample()`) work based on this index

In [10]:
df.set_index("Date", inplace = True)  # inplace=Ture: modifies the original df directly rather than returning a new df
df.head()

,Timestamp,Price
Date,,
2024-10-22,1729555200000,67394.857757
2024-10-23,1729641600000,67351.047935
2024-10-24,1729728000000,66683.968065
2024-10-25,1729814400000,68214.052052
2024-10-26,1729900800000,66585.665355


### 1.2 `cg.get_coin_market_chart_range_by_id()` API

- **:param `id`:** default is Bitcoin "bitcoin"
- **:param `vs_currency`:** default is USD "usd"
- **:param `from_timestamp`:** the Unix timestamp
- **:param `to_timestamp`:** the Unix timestamp

**If the range is less than 1 month, it will return hourly data, otherwise it will return daily data**

In [30]:
start_date = "2023-11-05"
end_date = "2023-12-01"

#### Convert start and end dates to Unix timestamps

In [12]:
pd.to_datetime(start_date)

Timestamp('2024-01-01 00:00:00')

In [13]:
pd.to_datetime(start_date).timetuple()

time.struct_time(tm_year=2024, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=1, tm_isdst=-1)

In [14]:
time.mktime(pd.to_datetime(start_date).timetuple())

1704085200.0

In [15]:
int(time.mktime(pd.to_datetime(start_date).timetuple()))

1704085200

In [31]:
from_timestamp = int(time.mktime(pd.to_datetime(start_date).timetuple()))
to_timestamp = int(time.mktime(pd.to_datetime(end_date).timetuple()))

#### Get the data

In [32]:
data = cg.get_coin_market_chart_range_by_id(id = "bitcoin",
                                            vs_currency = "usd",
                                            from_timestamp = from_timestamp,
                                            to_timestamp = to_timestamp)
type(data)

dict

In [33]:
df = pd.DataFrame(data["prices"], columns = ["Timestamp", "Price"])

df.head()                  

,Timestamp,Price
0,1699156829487,35285.450652
1,1699160417102,35194.722133
2,1699164036211,35180.420595
3,1699167623137,35082.736491
4,1699171251030,35204.898928


In [34]:
df["Date"] = pd.to_datetime(df["Timestamp"], unit = "ms")
df.tail()

,Timestamp,Price,Date
620,1701388860292,37711.818376,2023-12-01 00:01:00.292
621,1701392480275,37686.883494,2023-12-01 01:01:20.275
622,1701396035411,37952.976881,2023-12-01 02:00:35.411
623,1701399631877,37993.781302,2023-12-01 03:00:31.877
624,1701403225630,38056.214340,2023-12-01 04:00:25.630


In [35]:
df.set_index("Date", inplace=True)
df.head()

,Timestamp,Price
Date,,
2023-11-05 04:00:29.487,1699156829487,35285.450652
2023-11-05 05:00:17.102,1699160417102,35194.722133
2023-11-05 06:00:36.211,1699164036211,35180.420595
2023-11-05 07:00:23.137,1699167623137,35082.736491
2023-11-05 08:00:51.030,1699171251030,35204.898928


#### Using `resample()` to get 4-hourly data

In [26]:
df_4h = df.resample('4h').first()

df_4h.head()

,Timestamp,Price
Date,,
2023-11-05 04:00:00,1699156829487,35285.450652
2023-11-05 08:00:00,1699171251030,35204.898928
2023-11-05 12:00:00,1699185606559,35100.903834
2023-11-05 16:00:00,1699200024087,34913.588139
2023-11-05 20:00:00,1699214452413,35017.058006


### 1.3 fetch hourly data ranging more than 30 days

In [36]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import time

In [ ]:
start_date = "2024-01-08"

In [37]:
current_start = pd.to_datetime(start_date)
current_start

Timestamp('2023-11-05 00:00:00')

In [38]:
pd.DateOffset(days=30)

<DateOffset: days=30>

In [39]:
current_end = current_start + pd.DateOffset(days=30)
current_end

Timestamp('2023-12-05 00:00:00')

In [41]:
current_start < current_end

True

In [40]:
current_start.timestamp()

1699142400.0

In [ ]:
def fetch_hourly_data(coin_id, vs_currency, start_date, end_date):
    """
    """
    cg = CoinGeckoAPI()
    df_list = []

    current_start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    while current_start < end:
        current_end = current_start + pd.DateOffset(days = 30)

        if current_end > end:
            current_end = end

        from_timestamp = int(current_start.timestamp())
        to_timestamp = int(current_end.timestamp())
                             
        data = cg.get_coin_market_chart_range_by_id(id = coin_id, vs_currency = vs_currency,
                                                from_timestamp = from_timestamp, to_timestamp = to_timestamp)

        df = pd.DataFrame(data["prices"], columns = ["Timestamp", "Price"])
        df["Date"] = pd.to_datetime(df["Timestamp"], unit="ms")
        df.set_index("Date", inplace = True)

        df_list.append(df)

    return pd.concat(df_list)
                          
                            

In [44]:
df1 = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03'],
    'Close': [100, 101, 102]
})

df2 = pd.DataFrame({
    'Date': ['2023-01-04', '2023-01-05', '2023-01-06'],
    'Close': [103, 104, 105]
})

df3 = pd.DataFrame({
    'Date': ['2023-01-07', '2023-01-08', '2023-01-09'],
    'Close': [106, 107, 108]
})


df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])
df3['Date'] = pd.to_datetime(df3['Date'])

df1.set_index('Date', inplace=True)
df2.set_index('Date', inplace=True)
df3.set_index('Date', inplace=True)


df_list = [df1, df2, df3]
df_list

[            Close
 Date             
 2023-01-01    100
 2023-01-02    101
 2023-01-03    102,
             Close
 Date             
 2023-01-04    103
 2023-01-05    104
 2023-01-06    105,
             Close
 Date             
 2023-01-07    106
 2023-01-08    107
 2023-01-09    108]

In [45]:
pd.concat(df_list)

,Close
Date,
2023-01-01,100
2023-01-02,101
2023-01-03,102
2023-01-04,103
2023-01-05,104
2023-01-06,105
2023-01-07,106
2023-01-08,107
2023-01-09,108


### 1.4 CoinGecko Data Generator

In [2]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import time

def fetch_hourly_data(coin_id, vs_currency, start_date, end_date):
    """
    Fetch hourly cryptocurrency data using the range API from CoinGeckoAPI in chunks of 30 days

    :param coin_id: CoinGecko ID of the cryptocurrency (default is Bitcoin "bitcoin").
    :param vs_currency: The currency in which to get the market data (default is USD "usd").
    :param start_date: The start date for the data (format 'YYYY-MM-DD').
    :param end_date: The end date for the data (format 'YYYY-MM-DD').
    :return: A DataFrame
    """
    
    cg = CoinGeckoAPI()
    df_list = []

    current_start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    while current_start < end:
        current_end = current_start + pd.DateOffset(days = 30)

        if current_end > end:
            current_end = end

        from_timestamp = int(current_start.timestamp())
        to_timestamp = int(current_end.timestamp())

        print(f"fetching hourly data from {current_start} to {current_end}...")
        data = cg.get_coin_market_chart_range_by_id(id = coin_id, vs_currency = vs_currency,
                                                from_timestamp = from_timestamp, to_timestamp = to_timestamp)

        # convert data to a dataframe
        df = pd.DataFrame(data["prices"], columns = ["Timestamp", "Price"])
        df["Date"] = pd.to_datetime(df["Timestamp"], unit="ms")
        df.set_index("Date", inplace = True)

        df_list.append(df)

        # Add a delay between requests to avoid hitting the rate limit
        time.sleep(11)   # wait 21 seconds

        # update the current_start
        current_start = current_end

    
    # Concatenate all dataframes into one dataframe
    return pd.concat(df_list)


def generate_data_coingecko(output_path, coin_id = "bitcoin", vs_currency = "usd",
                            start_date = "2023-11-01", end_date = "2024-10-01",
                            interval = "4h"):
    """
    Fetch cryptocurrency data using the range API from CoinGeckoAPI,
    resample to the desired interval, and save it as a CSV file.

    :param output_path: Path to save the CSV file.
    :param coin_id: CoinGecko ID of the cryptocurrency (default is Bitcoin "bitcoin").
    :param vs_currency: The currency in which to get the market data (default is USD "usd").
    :param start_date: The start date for the data (format 'YYYY-MM-DD').
    :param end_date: The end date for the data (format 'YYYY-MM-DD').
    :param interval: Desired resampling interval (e.g., '4h' for 4-hour intervals).
    """

    # Fetch the hourly data using helper function
    df = fetch_hourly_data(coin_id, vs_currency, start_date, end_date)
    
    # Resample the data to desired interval
    df_resampled = df.resample(interval).first()

    # Save teh data as a CSV file to the output path
    df_resampled.to_csv(output_path)
    print(f"Data has been saved to {output_path} with {interval} intervals.")
 
    

if __name__ == "__main__":
    generate_data_coingecko("../data/bitcoin_4h_data.csv",coin_id = "bitcoin", vs_currency = "usd",
                            start_date = "2023-11-01", end_date = "2024-10-01",
                            interval = "4h")
    


fetching hourly data from 2023-11-01 00:00:00 to 2023-12-01 00:00:00...
fetching hourly data from 2023-12-02 00:00:00 to 2024-01-01 00:00:00...
fetching hourly data from 2024-01-02 00:00:00 to 2024-02-01 00:00:00...
fetching hourly data from 2024-02-02 00:00:00 to 2024-03-03 00:00:00...
fetching hourly data from 2024-03-04 00:00:00 to 2024-04-03 00:00:00...
fetching hourly data from 2024-04-04 00:00:00 to 2024-05-04 00:00:00...
fetching hourly data from 2024-05-05 00:00:00 to 2024-06-04 00:00:00...
fetching hourly data from 2024-06-05 00:00:00 to 2024-07-05 00:00:00...
fetching hourly data from 2024-07-06 00:00:00 to 2024-08-05 00:00:00...
fetching hourly data from 2024-08-06 00:00:00 to 2024-09-05 00:00:00...
fetching hourly data from 2024-09-06 00:00:00 to 2024-10-01 00:00:00...
Data has been saved to ../data/bitcoin_4h_data.csv with 4h intervals.


## 2. CoinCap

Can only get 1 month daily data

In [40]:
import requests
import pandas as pd

In [41]:
url = "https://api.coincap.io/v2/assets/ethereum/history?interval=d1"
response = requests.get(url)
data = response.json()

In [42]:
type(data)

dict

In [43]:
df = pd.DataFrame(data['data'])
df['date'] = pd.to_datetime(df['time'], unit='ms')
df.set_index('date', inplace=True)
df = df[['priceUsd']].rename(columns={'priceUsd': 'Price'})

In [44]:
df.head()

,Price
date,
2023-10-28,1788.0622255959847182
2023-10-29,1791.8593855739585031
2023-10-30,1804.0195782396389757
2023-10-31,1804.9241300449436976
2023-11-01,1814.1532215049560994


In [45]:
df.tail()

,Price
date,
2024-10-21,2702.4709225433300124
2024-10-22,2633.9876711020570245
2024-10-23,2565.1936050129203745
2024-10-24,2534.2717917868754643
2024-10-25,2509.4665188101942362


## 3. Yahoo Finance

In [33]:
import yfinance as yf
import pandas as pd

eth_data = yf.download('ETH-USD', start='2015-01-01', interval='1d')

[*********************100%***********************]  1 of 1 completed


In [34]:
eth_data.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
Date,,,,,,
2017-11-09 00:00:00+00:00,320.884003,320.884003,329.451996,307.056000,308.644989,893249984
2017-11-10 00:00:00+00:00,299.252991,299.252991,324.717987,294.541992,320.670990,885985984
2017-11-11 00:00:00+00:00,314.681000,314.681000,319.453003,298.191986,298.585999,842300992
2017-11-12 00:00:00+00:00,307.907990,307.907990,319.153015,298.513000,314.690002,1613479936
2017-11-13 00:00:00+00:00,316.716003,316.716003,328.415009,307.024994,307.024994,1041889984


In [35]:
eth_data.tail()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
Date,,,,,,
2024-10-21 00:00:00+00:00,2665.712158,2665.712158,2765.549561,2655.294434,2746.305664,17328615363
2024-10-22 00:00:00+00:00,2620.197510,2620.197510,2669.790039,2605.618652,2665.656738,15541911074
2024-10-23 00:00:00+00:00,2509.098877,2509.098877,2624.450439,2457.169922,2620.088623,17876984551
2024-10-24 00:00:00+00:00,2534.498535,2534.498535,2559.151367,2506.722900,2523.606934,16128627601
2024-10-26 00:00:00+00:00,2450.075439,2450.075439,2450.075439,2428.771240,2435.932861,24082782208


In [36]:
isinstance(eth_data.columns, pd.MultiIndex)

True

In [37]:
eth_data.columns = eth_data.columns.get_level_values(0)

In [38]:
eth_data.head()

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2017-11-09 00:00:00+00:00,320.884003,320.884003,329.451996,307.056000,308.644989,893249984
2017-11-10 00:00:00+00:00,299.252991,299.252991,324.717987,294.541992,320.670990,885985984
2017-11-11 00:00:00+00:00,314.681000,314.681000,319.453003,298.191986,298.585999,842300992
2017-11-12 00:00:00+00:00,307.907990,307.907990,319.153015,298.513000,314.690002,1613479936
2017-11-13 00:00:00+00:00,316.716003,316.716003,328.415009,307.024994,307.024994,1041889984


In [39]:
eth_data.columns

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

### 3.1 Yahoo Finance Data Generator

In [ ]:
import yfinance as yf
import pandas as pd

def generate_data_yf(output_path, symbol = "ETH-USD",
                     start_date = "2017-01-01", end_date = None,
                     interval = "1d"):
    """
    Fetch cryptocurrency data from Yahoo Finance API, and save it as a CSV file.

    :param output_path: the path where the csv file will be saved
    :param symbol: defauls symbol is ETH-USD
    :param start_date: Start Date (YYYY-MM-DD)
    :param end_date: End Date (YYYY-MM-DD)
    :param interval: time interval ("1d" for daily, "1h" for hourly"
    """

    # If end_date is None, don't pass the end argument, which defaults to today
    if end_date is None:
        
        print(f"fecthing {symbol} data from {start_date} to today with interval {interval}...")
        data = yf.download(symbol, start = start_date, 
                           interval =i nterval)
    else:
        
        print(f"fecthing {symbol} data from {start_date} to {end_date} with interval {interval}...")
        data = yf.download(symbol, 
                           start = start_date, end = end_date, 
                           interval = interval)
    
    # Check if data was successfully fetched
    if data.empty:
        print(f"No data found for {symbol} with the given time range")
        return

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    data.to_csv(output_path)
    print(f"data has been saved to {output_path}")


# Save the data to a CSV file
if __name__ == "__main__":
    
    generate_data_yf("../data/eth_daily_data.csv", symbol = "ETH-USD",
                     start_date = "2017-01-01", end_date = None,
                     interval = "1d")



## 4. csv_loader.py

In [2]:
!ls ../data

bitcoin_4h_data.csv eth_daily_data.csv


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/bitcoin_4h_data.csv")
df.head()

,Date,Timestamp,Price
0,2023-11-02 00:00:00,1.698883e+12,35457.454912
1,2023-11-02 04:00:00,NaN,NaN
2,2023-11-02 08:00:00,NaN,NaN
3,2023-11-02 12:00:00,NaN,NaN
4,2023-11-02 16:00:00,NaN,NaN


In [5]:
df = pd.read_csv("../data/bitcoin_4h_data.csv", index_col = "Date")
df.head()


,Timestamp,Price
Date,,
2023-11-02 00:00:00,1.698883e+12,35457.454912
2023-11-02 04:00:00,NaN,NaN
2023-11-02 08:00:00,NaN,NaN
2023-11-02 12:00:00,NaN,NaN
2023-11-02 16:00:00,NaN,NaN


In [6]:
df.tail()

,Timestamp,Price
Date,,
2024-09-30 08:00:00,NaN,NaN
2024-09-30 12:00:00,NaN,NaN
2024-09-30 16:00:00,NaN,NaN
2024-09-30 20:00:00,NaN,NaN
2024-10-01 00:00:00,1.727741e+12,63243.275325
